## **Neural Architecture Search (NAS)**

Neural Architecture Search (NAS) is an advanced technique in deep learning where the architecture of the neural network is automatically searched and optimized to achieve the best performance for a specific task. NAS aims to find the most efficient neural network architecture, typically through reinforcement learning or evolutionary algorithms, to minimize human intervention and design time.

**Imports**

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

**Data Loading**

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Minimal Preprocessing**

In [ ]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

**Neural Architecture Search Function**

In [ ]:
def create_model():
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))
    # Randomly define the number of layers and units per layer
    num_layers = np.random.randint(1, 4)
    for _ in range(num_layers):
        units = np.random.randint(32, 128)  # Random number of units per layer
        model.add(layers.Dense(units, activation='relu'))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

**Model Training**

In [ ]:
best_model = None
best_accuracy = 0

for _ in range(5):  # Try 5 random architectures
    model = create_model()
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model

**Predictions**

In [ ]:
y_pred_prob = best_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

**Performance Metrics**

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy of Best Model: {accuracy:.2f}")

**Visualizations**

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show() 

**Model Visualization**

In [ ]:
plot_model(best_model, to_file='best_model.png', show_shapes=True, show_layer_names=True)